In [ ]:
from glob import glob
import shutil
from PIL import Image
import os
from random import shuffle
from tqdm import tqdm
import cv2
import numpy as np
def createDirectory(directory):
    """_summary_
        create Directory
    Args:
        directory (string): file_path
    """    
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")


In [ ]:
import matplotlib.pyplot as plt
img=cv2.imread('../../data/normalization_type/NIA6_R_BRDC_BRCA-AJ-00158-S-TP-02_42_20.jpeg')
mask=np.zeros((img.shape[0],img.shape[1]),dtype=np.uint8)
mask=np.where(img.std(axis=2)<3,255,0)
Image.fromarray(mask.astype(np.uint8)).save('../../data/normalization_type/NIA6_R_BRDC_BRCA-AJ-00158-S-TP-02_42_20_mask.jpeg')


In [ ]:
file_list=glob('../../data/origin_type/**/**/*.jpeg')
save_list=[f.replace('/origin_type', '/Image_down/origin') for f in file_list]
for j in tqdm(range(len(file_list))):
    createDirectory(os.path.dirname(save_list[j]))
    shutil.copy(file_list[j],save_list[j])

In [ ]:
file_list=glob('../../data/synth/json/**/**/*.jpeg')
save_list=[f.replace('/synth/json/_', '/Image_down/synth/') for f in file_list]
shuffle(save_list)
for j in tqdm(range(15970)):
    createDirectory(os.path.dirname(save_list[j]))
    shutil.copy(file_list[j],save_list[j])

In [ ]:
carcinoma='STMX'
Refined_file_list=glob('../../data/origin_type/'+carcinoma+'/**/*.jpeg')
Refined_wsi_list=list(set([os.path.basename(f).split('_')[3] for f in Refined_file_list]))
Whole_file_list=glob('../../data/NIA/'+carcinoma+'/*.jpeg')
Whole_wsi_list=list(set([os.path.basename(f).split('_')[3] for f in Whole_file_list]))
print(len(Whole_wsi_list),len(Refined_wsi_list),len(Whole_file_list),len(Refined_file_list))

In [ ]:
path_list=glob('../../result/synth/**/**/*.jpeg')
for i in range(len(path_list)):
    file_name=path_list[i].replace('/result/synth/','/result/synth_1/')
    shutil.move(path_list[i],file_name)
    

In [ ]:
import torch
import torchvision
topilimage = torchvision.transforms.ToPILImage()
topilimage(torch.randn((3,1024,1024)))

In [ ]:

import os
from datasets import load_dataset
from PIL import Image
import requests
from io import BytesIO
import pandas as pd

# 로컬 폴더 경로 설정
image_save_dir = "../../../colon_smathealthcare/Day5/notices/Day5_Session2/NLMCXR/nlmcxr_images/"
label_save_file = "../../../colon_smathealthcare/Day5/notices/Day5_Session2/NLMCXR/nlmcxr_labels.csv"

# 폴더가 존재하지 않으면 생성
os.makedirs(image_save_dir, exist_ok=True)

# 데이터셋 다운로드
ds = load_dataset("Fakhraddin/NLMCXR")
df=pd.DataFrame(columns=['image','text'])
# 이미지와 라벨 저장
with open(label_save_file, 'w', encoding='utf-8') as label_file:
    for i, sample in enumerate(ds['train']):
        image_url = sample['image']  # 이미지 URL 또는 경로 (데이터셋 구조에 맞게 수정)
        label = sample['text']      # 라벨 (데이터셋 구조에 맞게 수정)
        file_name=os.path.basename(sample['path'])
        # 이미지 다운로드 및 저장
        
        image_url.save(image_save_dir+file_name)
            # 라벨 저장
        df.loc[i]=[file_name,label]

df.to_csv(label_save_file, index=False)


In [ ]:
df_sampled = df.sample(n=1000, random_state=42)
df_sampled.to_csv(label_save_file, index=False) 

In [24]:
def process_image(image_path, output_size=(256, 256)):
    try:
        img = Image.open(image_path)
        img.thumbnail(output_size, Image.Resampling.LANCZOS)  # 비율 유지하여 축소
        new_img = Image.new("RGB", output_size, (0, 0, 0))  # 패딩용 검은색 배경
        new_img.paste(img, ((output_size[0] - img.size[0]) // 2, (output_size[1] - img.size[1]) // 2))
        return new_img
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return None
output_directory='../../../colon_smathealthcare/Day5/notices/Day5_Session2/NLMCXR/nlmcxr_images_down/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for index, row in df_sampled.iterrows():
    image_path = os.path.join(image_save_dir, row['image'])
    processed_img = process_image(image_path)
    if processed_img:
        output_path = os.path.join(output_directory, row['image'])
        processed_img.save(output_path)

print("Image processing complete.")

Image processing complete.
